In [19]:
import os
import sys

In [20]:
cd C:\TextSummarizerProject

C:\TextSummarizerProject


In [21]:
%pwd

'C:\\TextSummarizerProject'

In [22]:
## UPDATE ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    ALL_REQUIRED_FILES:list
    

In [23]:
## UPDATE CONFIGURATION MANAGER

from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories




In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

       
        self.artifacts_root = self.config.get("artifacts_root")

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.get("data_validation")  
        create_directories([config["root_dir"]])

        data_validation_config = DataValidationConfig(
            root_dir=config["root_dir"],
            STATUS_FILE=config["STATUS_FILE"],
            ALL_REQUIRED_FILES=config["ALL_REQUIRED_FILES"]
        )

        return data_validation_config


In [25]:
#COMPONENT

import os
from textsummarizer.logging import logger

In [26]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            all_files = os.listdir(os.path.join("artifacts", "data_ingestion", "samsum_dataset"))
            print("All Files:", all_files)

            
            valid_files = [file for file in all_files if file.split('.')[0] in self.config.ALL_REQUIRED_FILES]
            print("Valid Files:", valid_files)

            validation_status = len(valid_files) == len(self.config.ALL_REQUIRED_FILES)

            print("Validation Status:", validation_status)

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e


In [28]:
#PIPELINE

try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)

    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2023-08-29 14:11:00,204:INFO:common:yaml file: config\config.yaml loaded successfully]
[2023-08-29 14:11:00,204:INFO:common:yaml file: params.yaml loaded successfully]
[2023-08-29 14:11:00,208:INFO:common:created directory at: artifacts/data_validation]
All Files: ['dataset_dict.json', 'test', 'train', 'validation']
Valid Files: ['test', 'train', 'validation']
Validation Status: True
